# What’s New In Python 3.10

---

## 1. Pattern Matching

One of the most significant features in Python 3.10 is the introduction of Structural Pattern Matching. It allows you to match patterns in an object structure using a new `match` statement, which is similar to the `switch` or `case` statements found in other languages.


```python
match subject:
    case <pattern_1>:
        <action_1>
    case <pattern_2>:
        <action_2>
    case _:
        <action_default>
```

### Example 1:

- Using `if` statement

In [1]:
def http_error(status):
    if status == 400:
        return "Bad request"
    elif status == 404:
        return "Not found"
    elif status == 418:
        return "I'm a teapot"
    else:
        return "Something's wrong with the internet"

print(http_error(400))
print(http_error(404))
print(http_error(418))
print(http_error(500))


Bad request
Not found
I'm a teapot
Something's wrong with the internet


- Using `match-case` pattern

In [2]:
def http_error(status):
    match status:
        case 400:
            return "Bad request"
        case 404:
            return "Not found"
        case 418:
            return "I'm a teapot"
        case _:
            return "Something's wrong with the internet"

print(http_error(400))
print(http_error(404))
print(http_error(418))
print(http_error(500))


Bad request
Not found
I'm a teapot
Something's wrong with the internet


### Example 2:

In [3]:
def greet(person):
    match person:
        case {'name': 'Alice', 'age': age} if age < 30:
            return f"Hello, young Alice!"
        case {'name': 'Bob', 'age': 30}:
            return "Hello, Bob!"
        case {'name': name}:
            return f"Hello, {name}!"
        case _:
            return "Who are you?"

print(greet({'name': 'Alice', 'age': 25}))
print(greet({'name': 'Bob', 'age': 30}))
print(greet({'name': 'Charlie'}))
print(greet('Unknown'))


Hello, young Alice!
Hello, Bob!
Hello, Charlie!
Who are you?


### Example 3:

In [4]:
dict_1 = {
    "id": 1,
    "meta": {
        "source": "path1",
        "location": "west",
    }
}

dict_2 = {
    "id": 2,
    "source": "path2",
    "location": "east",
}

for d in (dict_1, dict_2, "other"):
    match d:
        case {
            "id": ident,
            "meta": {
                "source": source,
                "location": loc,
                }
            }:
            print(ident, source, loc)

        case {
            "id": ident,
            "source": source,
            "location": loc,
            }:
            print(ident, source, loc)

        case _:
            print("No Match")

1 path1 west
2 path2 east
No Match


## 2. Parenthesized context managers

In Python 3.10, it is now possible to use multiple context managers in a single `with` statement without backslashes. The new syntax uses parentheses for a cleaner and more readable way.

In [5]:
# Old Version
with open("file1.txt", "w") as f_in,\
     open("file2.txt", "w") as f_out:
    pass


In [6]:
# New Version
with (
    open("file1.txt", "w") as f_in,
    open("file2.txt", "w") as f_out
):
    pass

## 3. More precise types

Python 3.10 introduced several improvements and new features to the `typing` module. Some of the key additions include:

### 3.1 Allow writing union types as `X | Y`

This PEP proposes overloading the `|` operator on types to allow writing `Union[X, Y]` as `X | Y`, and allows it to appear in `isinstance` and `issubclass` calls.

In [7]:
# Old Version
from typing import Union

def foo(value: Union[int, str]) -> None:
    if isinstance(value, int):
        print(value + 1)
    else:
        print(value.upper())

foo(5)
foo("Hi")

6
HI


In [8]:
# New Version
def foo(value: int | str) -> None:
    if isinstance(value, int):
        print(value + 1)
    else:
        print(value.upper())

foo(5)
foo("Hi")

6
HI


### 3.2 Parameter Specification Variables:

`typing.ParamSpec` improve the type hinting story in Python, especially for higher-order functions and decorators. It provides a way to accurately represent callable parameter signatures in type annotations, thus helping developers and tooling maintain type safety across function boundaries.

Suppose you're writing a decorator that wraps a function, without modifying its signature. Prior to PEP 612, you could use `typing.Callable` to specify that your parameter is a function, but you could not specify the signature of that function.

- Without `ParamSpec`

In [9]:
from typing import Callable

def my_decorator(f: Callable) -> Callable:
    def wrapper(*args, **kwargs):
        # Do something before
        result = f(*args, **kwargs)
        # Do something after
        return result
    return wrapper

@my_decorator
def greet(name: str) -> str:
    return f"Hello, {name}"


In this code, we're using `Callable`, but it doesn't retain information about the `greet` function's specific argument types and return type. The type checker doesn't know that `wrapper` and `greet` have the same signature.

- With `ParamSpec`

In [10]:
from typing import Callable, TypeVar, ParamSpec

P = ParamSpec('P')
R = TypeVar('R')

def my_decorator(f: Callable[P, R]) -> Callable[P, R]:
    def wrapper(*args: P.args, **kwargs: P.kwargs) -> R:
        # Do something before
        result = f(*args, **kwargs)
        # Do something after
        return result
    return wrapper

@my_decorator
def greet(name: str) -> str:
    return f"Hello, {name}"


Using `ParamSpec`, the type checker knows that the `wrapper` function takes the same parameters as the `greet` function. This makes type checking more accurate and useful for decorators or any higher-order function that wraps another callable.

Here's the breakdown of the updated code:
- `P = ParamSpec('P')` defines a new `ParamSpec` variable called `P`. It's used to represent the function's parameters.
- `R = TypeVar('R')` defines a type variable for the return type.
- In `my_decorator`, the type `Callable[P, R]` specifies that the decorator takes a callable whose parameters and return type are described by `P` and `R`, respectively.
- The type of `wrapper` is marked with `*args: P.args` and `**kwargs: P.kwargs`, which means it should accept and pass through any arguments and keyword arguments acceptable by the function `f`.

With the help of `ParamSpec`, type checkers can now understand that `my_decorator` preserves the signature of the function it decorates, and `wrapper` will be treated as if it has the exact same parameters as `greet`. This way, any type annotations and checks will be accurately carried forward.

### 3.3 Explicit Type Aliases

PEP 613 introduces explicit type aliasing in Python via a new construct called `TypeAlias`. Before the introduction of PEP 613, type aliases in Python were defined using variable annotations, but there was no explicit way to indicate that a particular annotation was intended as a type alias. This could lead to confusion for both human readers and static type checkers.

- Without `TypeAlias`

In [11]:
from typing import List, Union

# Define type aliases using variable annotations.
Vector = List[float]
Result = Union[int, float]

def scale(scalar: float, vector: Vector) -> Vector:
    return [scalar * num for num in vector]

def get_result(value: Result) -> Result:
    return value * 2

- With `TypeAlias`

In [12]:
from typing import TypeAlias

# Explicitly define type aliases.
Vector: TypeAlias = list[float]
Result: TypeAlias = int | float

def scale(scalar: float, vector: Vector) -> Vector:
    return [scalar * num for num in vector]

def get_result(value: Result) -> Result:
    return value * 2

### 3.4 User-Defined Type Guards
`TypeGuard` is a special kind of type hint that can be used to indicate to type checkers that a particular condition provides a guarantee about the type of a variable.

- without `TypeGuard`

In [13]:
from typing import List, Union

def is_all_integers(lst: List[Union[int, str]]) -> bool:
    return all(isinstance(item, int) for item in lst)


def process_items(items: List[Union[int, str]]):
    if is_all_integers(items):
        # Even though we have confirmed that `items` is a list of integers,
        # static type checkers do not understand that, and treat `items` as a list
        # of Union[int, str] inside this branch.
        total = sum(items)  # This line will raise a type checking error
        print(f"The total is: {total}")
    else:
        print("Not all items are integers.")


Static type checkers will not be able to infer that `items` is a list of integers in the `if` block after the `is_all_integers` call; hence, they will flag an error on the `sum(items)` since `sum` expects an iterable of numbers, not strings.

- with `TypeGuard`

In [14]:
from typing import List, Union, TypeGuard

def is_all_integers(lst: List[Union[int, str]]) -> TypeGuard[List[int]]:
    return all(isinstance(item, int) for item in lst)

def process_items(items: List[Union[int, str]]):
    if is_all_integers(items):
        # Now, static type checkers understand that if `is_all_integers(items)` is True,
        # `items` is a list of integers inside this branch.
        total = sum(items)  # The type of 'items' is inferred as List[int] by type checkers
        print(f"The total is: {total}")
    else:
        print("Not all items are integers.")


With `TypeGuard`, you can now provide the guarantee to type checkers that `is_all_integers` narrows down the type of `lst` to `List[int]` if it returns `True`. As a result, static type checkers will understand the type-narrowing effect of the predicate function and will not raise an error on `sum(items)` in the `if` block. This allows for stronger type hinting and helps eliminate false-positive type errors in statically checked code.

## 4. Deprecations

Python 3.10 has started to deprecate old features like `typing.io` and `typing.re`, which will be removed in future versions. Instead, you should use the `io` and `re` modules respectively.